# A simple Python implementation of the ReAct pattern for LLMs

A popular doom scenario for AI is giving it access to tools, so it can make function calls, execute its own code and generally break free of the constraints of its initial environment.

Let's try that now!

The ReAct pattern (for Reason+Act) is described in this [paper](https://react-lm.github.io/). It's a popular pattern where you implement additional actions that an LLM can take - searching Wikipedia or running calculations for example - and then teach it how to request that those actions are run, then feed their results back into the LLM. Hence we have here an AI System which uses the AI component of a LLM but also traditional api and function calls. The AI just directed when to call which functions.

In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

This example agent should research for weather conditions for a few cities the author is frequently in and what activities you can do there plus some recommendations based on the weather

Let's first create the tools. The weather tool and the suggestion tool. These are just regular python functions with definition and input and output.

Here we do not use a regular weather api, but for simplicity a dummy one. And also a simplified activity recommendation. Of course these could be replaced with real counterparts for a production app.

In [44]:
import random
import ast

def get_weather(city):
    """
    Returns dummy weather data for a specified city.
    If the city is not supported, returns an error message.
    """
    AVAILABLE_CITIES = ["Kiel", "Hamburg", "Lübeck", "Berlin", "Munich"]
    WEATHER_CONDITIONS = ["Sunny", "Cloudy", "Rainy", "Windy", "Snowy"]

    # Check if the city is in the supported list
    if city not in AVAILABLE_CITIES:
        return {
            "message": f"City '{city}' not found. Available cities: {', '.join(AVAILABLE_CITIES)}"
        }

    # Generate random weather data
    temperature = random.randint(-5, 30)   # from -5 to 30 degrees
    humidity = random.randint(20, 90)      # from 20% to 90% humidity
    condition = random.choice(WEATHER_CONDITIONS)

    return {
        "city": city,
        "temperature": temperature,
        "humidity": humidity,
        "condition": condition
    }


def suggest_activities(weather_data):
    """
    Takes the output of get_weather() and returns a list of activities
    that fit the city's usual attractions and current weather conditions.

    If the input contains an error (i.e., 'message' key), returns that error.
    """
    # Convert the string to an actual dictionary
    weather_data = ast.literal_eval(weather_data)
    # If weather_data is actually an error message (unknown city), return it as is
    if "message" in weather_data:
        return {
            "error": weather_data["message"]
        }

    city = weather_data["city"]
    condition = weather_data["condition"]

    # Here we define two simple mappings:
    # 1. city-based activities
    # 2. condition-based activities
    # We then combine them to get the final list of suggestions.

    city_activities_map = {
        "Kiel":    ["Visit Laboe Naval Memorial", "Take a ferry ride on the Kiel Fjord"],
        "Hamburg": ["Check out the Elbphilharmonie", "Take a harbor tour"],
        "Lübeck":  ["Tour the old town", "Visit Holstentor"],
        "Berlin":  ["Visit the Brandenburg Gate", "Explore Museum Island"],
        "Munich":  ["Visit Marienplatz", "Stroll through the English Garden"]
    }

    condition_activities_map = {
        "Sunny":  ["Enjoy a cold drink outside", "Go for a walk in the park"],
        "Cloudy": ["Visit a museum", "Relax in a cozy cafe"],
        "Rainy":  ["Explore indoor attractions", "Watch a movie at the cinema"],
        "Windy":  ["Find an indoor event", "Check out a local show"],
        "Snowy":  ["Check out winter markets (if open)", "Enjoy a warm drink in a cafe"]
    }

    city_based_suggestions = city_activities_map.get(city, [])
    weather_based_suggestions = condition_activities_map.get(condition, [])

    # Combine both sets of suggestions into one list
    suggestions = city_based_suggestions + weather_based_suggestions

    return {
        "city": city,
        "condition": condition,
        "activities": suggestions
    }


# Example usage:

# 1. Known city
weather_info = get_weather("Hamburg")
print("Weather Info:", weather_info)
activities = suggest_activities(str(weather_info))
print("Suggested Activities:", activities)

# 2. Unknown city
weather_info = get_weather("New York")
print("Weather Info:", weather_info)
activities = suggest_activities(str(weather_info))
print("Suggested Activities:", activities)


Weather Info: {'city': 'Hamburg', 'temperature': -2, 'humidity': 76, 'condition': 'Rainy'}
Suggested Activities: {'city': 'Hamburg', 'condition': 'Rainy', 'activities': ['Check out the Elbphilharmonie', 'Take a harbor tour', 'Explore indoor attractions', 'Watch a movie at the cinema']}
Weather Info: {'message': "City 'New York' not found. Available cities: Kiel, Hamburg, Lübeck, Berlin, Munich"}
Suggested Activities: {'error': "City 'New York' not found. Available cities: Kiel, Hamburg, Lübeck, Berlin, Munich"}


Let's now write the prompt the elicit the ReAct behaviour in a LLM. Different LLMs reponds better of worse when asked to behave in this way. Newer models normally much better. In this notebook we wil use gpt-4o-mini.

In [54]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_weather:
e.g. get_weather: Kiel
Returns weather data for a specified city. If the city is not supported, returns an error message.

suggest_activities:
e.g. suggest_activities: {'city': 'Hamburg', 'temperature': 11, 'humidity': 51, 'condition': 'Sunny'}
Always use this to suggest activities for a given city and weather. Takes the output of get_weather and returns a list of activities that fit the city's usual attractions and current weather conditions.

Example sessions

Example session:

Question: What is today in Kiel?
Thought: I should look up the weather in Kiel using get_weather
Action: get_weather: Kiel
PAUSE

You will be called again with this:

Observation: {'city': 'Kiel', 'temperature': 3, 'humidity': 20, 'condition': 'Sunny'}

You then output:

Answer: The weather in Kiel is currently sunny with a temperature of 3°C. Humidity is quite low at 20%, making the air feel crisp and dry. It's a clear and bright day—perfect for outdoor activities if you dress warmly!
""".strip()

Let's now setup our Agent in python

In [4]:
import openai
import re
from openai import OpenAI
api_key = "" # Set to your api key e.g. "sk-..."

In [5]:
client = OpenAI(api_key = api_key)

In [6]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o-mini",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content


In [10]:
# initialize agent
react_agent = Agent(prompt)

In [11]:
# call agent
result = react_agent("What is the Weather in Munich?")
print(result)

Thought: I need to check the current weather in Munich using the get_weather action. 
Action: get_weather: Munich
PAUSE


As a starter let's parse the action and action input manually and hard code it for the function call. Later we wil automate this.

In [12]:
action_result = get_weather("Munich") # Munich was parsed and inserted by the human mind of the author

In [13]:
action_result

{'city': 'Munich', 'temperature': 11, 'humidity': 48, 'condition': 'Snowy'}

In [14]:
# create next prompt
follow_up_prompt = f"Observation: {action_result}"

In [15]:
follow_up_prompt

"Observation: {'city': 'Munich', 'temperature': 11, 'humidity': 48, 'condition': 'Snowy'}"

In [16]:
# call agent with the obervation
react_agent(follow_up_prompt)

"Answer: The weather in Munich is currently snowy with a temperature of 11°C. The humidity is at 48%, which can make the cold feel a bit more biting. It's a good day to enjoy some winter activities or cozy up indoors!"

In [17]:
react_agent.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_weather:\ne.g. get_weather: Kiel\nReturns weather data for a specified city. If the city is not supported, returns an error message.\n\nsuggest_activities:\ne.g. suggest_activities: {'city': 'Hamburg', 'temperature': 11, 'humidity': 51, 'condition': 'Sunny'}\nTakes the output of get_weather() and returns a list of activities that fit the city's usual attractions and current weather conditions.\n\nExample sessions\n\nExample session:\n\nQuestion: What is today in Kiel?\nThought: I should look up the weather in Kiel using get_weather\nAction: get_weather: Kiel\nPAUSE\n\nYou will be called again with this:\n

Let's try if the agent can also recommend something given the weather even if that was not given in our example sessions in the prompt string

In [30]:
# first reset our agent
react_agent = Agent(prompt)

In [31]:
react_agent.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_weather:\ne.g. get_weather: Kiel\nReturns weather data for a specified city. If the city is not supported, returns an error message.\n\nsuggest_activities:\ne.g. suggest_activities: {'city': 'Hamburg', 'temperature': 11, 'humidity': 51, 'condition': 'Sunny'}\nUse this to suggest activities for a given city and weather. Takes the output of get_weather and returns a list of activities that fit the city's usual attractions and current weather conditions.\n\nExample sessions\n\nExample session:\n\nQuestion: What is today in Kiel?\nThought: I should look up the weather in Kiel using get_weather\nAction: get_we

In [32]:
result = react_agent("What activities can you recommend today in Kiel?")
print(result)

Thought: To recommend activities in Kiel, I first need to check the current weather conditions there. I'll use the get_weather action to gather that information.  
Action: get_weather: Kiel  
PAUSE


In [33]:
action_result = get_weather("Kiel")

In [34]:
action_result

{'city': 'Kiel', 'temperature': 19, 'humidity': 50, 'condition': 'Sunny'}

In [35]:
follow_up_prompt = f"Observation: {action_result}"

In [ ]:
follow_up_result = react_agent(follow_up_prompt)

In [37]:
follow_up_result

"Thought: The weather in Kiel is sunny with a pleasant temperature of 19°C and moderate humidity at 50%. This kind of weather is ideal for various outdoor activities. I will now suggest activities that are suitable for this weather.  \nAction: suggest_activities: {'city': 'Kiel', 'temperature': 19, 'humidity': 50, 'condition': 'Sunny'}  \nPAUSE"

In [38]:
action_result = suggest_activities("{'city': 'Kiel', 'temperature': 19, 'humidity': 50, 'condition': 'Sunny'}")

In [39]:
action_result

{'city': 'Kiel',
 'condition': 'Sunny',
 'activities': ['Visit Laboe Naval Memorial',
  'Take a ferry ride on the Kiel Fjord',
  'Enjoy a cold drink outside',
  'Go for a walk in the park']}

In [40]:
follow_up_prompt = f"Observation: {action_result}"

In [41]:
follow_up_result = react_agent(follow_up_prompt)

In [42]:
follow_up_result

'Answer: Today in Kiel, with sunny weather and a comfortable temperature of 19°C, I recommend the following activities: \n\n1. Visit the Laboe Naval Memorial, which offers historical insights and great views.\n2. Take a ferry ride on the Kiel Fjord for a scenic experience on the water.\n3. Enjoy a cold drink outside at one of the local cafes or bars.\n4. Go for a walk in the park to enjoy the beautiful weather and greenery.\n\nThese activities will allow you to make the most of the lovely day!'

So even without an example just given the appropriate description in the prompt the model can generalize.

**Exercise:** Make the description of **suggest_activities** more blurry and see if the agent still calls it

So currently we read the answer manually from the result string. We will automate this now:

In [43]:
# First we need a regular expression to strip out the name of the action and the input
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [48]:
# We create a simple dict to save which actions are available
known_actions = {
    "get_weather": get_weather,
    "suggest_activities": suggest_activities
}

In [55]:
def query(question, max_turns=5):
    i = 0
    react_agent = Agent(prompt) # initialize ReAct Agent
    follow_up_prompt = question
    while i < max_turns:
        i += 1
        result = react_agent(follow_up_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(f" -- running {action} {action_input}")
            # run selected action with action input to get the obeservation
            observation = known_actions[action](action_input.strip())
            print("Observation:", observation)
            follow_up_prompt = f"Observation: {observation}"
        else:
            return

In [56]:
question = "What activies can you recommend today in Berlin"
query(question)

Thought: I need to check the current weather in Berlin to suggest appropriate activities. I'll use the get_weather action for that.  
Action: get_weather: Berlin  
PAUSE
 -- running get_weather Berlin  
Observation: {'city': 'Berlin', 'temperature': 26, 'humidity': 52, 'condition': 'Rainy'}
Thought: The weather in Berlin is rainy with a temperature of 26°C and a humidity level of 52%. This suggests that outdoor activities might not be ideal, but there could be some interesting indoor options. I will now suggest activities based on this weather condition.  
Action: suggest_activities: {'city': 'Berlin', 'temperature': 26, 'humidity': 52, 'condition': 'Rainy'}  
PAUSE
 -- running suggest_activities {'city': 'Berlin', 'temperature': 26, 'humidity': 52, 'condition': 'Rainy'}  
Observation: {'city': 'Berlin', 'condition': 'Rainy', 'activities': ['Visit the Brandenburg Gate', 'Explore Museum Island', 'Explore indoor attractions', 'Watch a movie at the cinema']}
Answer: Today in Berlin, with 

This concludes our python implementation of the react pattern. You can see that is does not include a lot of code. Here mainly the idea matters. For futher readings go through our in depth session and read the original paper

## Appendix

The overall prompt design is not really specific; the idea of the pattern Thought, Action, Oberservation is the important idea/core. In this notebook the PAUSE is just introduced that the LLM stops its generation and that works quite well. So after it stops we can then parse the action instructions. If I would not stop then we would have a big problem. For the original paper the authors used GPT3 and thus had not this oppertunitiy hence they choose in the api call to GPT3 the stop parameter: *stop=[f"\nObservation {i}:"]*. So the model thought it should generate everything on its own and is by this trick it was programmatically interrupted in its generation. The rest is as above. One prompt the ReAct authors use is for example:

"""

Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types:

(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.

(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.

(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.

"""

As you can see there is no PAUSE. In general what you should take away is that it is not about the specific prompt design but more the overall pattern of thought, action and observation which yields huge benefits.